In [ ]:
!pip install 'transformers[torch]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 11.4 MB/s eta 0:00:00


In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
PROJECT_ROOT = f"/content/gdrive/MyDrive/GPT-VLU"
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
sys.path.insert(0, PROJECT_ROOT)

In [ ]:
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer
from transformers import pipeline, set_seed

from utils.setup_utils import parse_and_validate_df, DFExperimentInfo

import pandas as pd
import numpy as np
from dataclasses import fields
import torch
from sklearn.metrics import accuracy_score

from tqdm import tqdm

In [ ]:
# Loading data
dataset_path = '/content/gdrive/MyDrive/GPT-VLU/datasets/color-concrete-objects.csv'

task = 'choice'
df = pd.read_csv(dataset_path)
df.head()

,object,color
0,ash,grey
1,banana,yellow
2,beaver,brown
3,blood,red
4,broccoli,green


In [ ]:
df['color'].unique()

array(['grey', 'yellow', 'brown', 'red', 'green', 'orange', 'white',
       'black', 'pink', 'blue'], dtype=object)

In [ ]:
color_options = df['color'].unique()

In [ ]:
len(color_options)

10

## Initialize the model - GPT2

In [ ]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
set_seed(42)
feature_extractor = pipeline('feature-extraction', model='gpt2')

In [ ]:
generator_large = pipeline('text-generation', model='gpt2-large')
set_seed(42)

In [ ]:
feature_extractor_large = pipeline('feature-extraction', model='gpt2-large')

## Colors

TODO: I can test without restricting the size of the outcome.

Possible extra prompts for this:
1. ... instead of MASK, should be 
2. ... 
3. ... fill in the MASK 
4. ... choose one option from the list: <> to replace MASK 
5. ... replace the MASK with correct word
6. ...

In [ ]:
colors_orig_prompts = [
    'A picture of a MASK WORD',
    'A photo of a MASK WORD',
    'A photo of the MASK WORD',
    'A MASK WORD',
    'the MASK WORD',
    'The normal color of a WORD is MASK',
    'WORD usually has a MASK color',
    'WORDs have a color',
    'What is a color of a WORD? MASK',
    'The natural color of a WORD is MASK' 
  ]

In [ ]:
def score_model(fe_model, orig_prompts):
  y_true = []
  y_pred = []

  for index, row in tqdm(df.iterrows()):
    w = row['object']
    for p in orig_prompts:
      p = p.replace("WORD", w)

      options_results = [] 
      for option in color_options:
        input_prompt = p.replace("MASK", option)
        v = torch.tensor(fe_model(input_prompt, max_new_tokens=0, 
                                  num_return_sequences=0, pad_token_id=50256)
                                  ).squeeze().mean(axis=0).squeeze()
        v /= v.norm(p=2, dim=-1, keepdim=True)
        v = v.cpu().numpy()
        options_results.append(v)

      base = torch.tensor(fe_model(p.replace("MASK", ''), 
                                   max_new_tokens=0, num_return_sequences=0, 
                                   pad_token_id=50256)).squeeze().mean(axis=0).squeeze()
      base /= base.norm(p=2, dim=-1, keepdim=True)
      base = base.cpu().numpy()

      scores = np.array(options_results) @ np.expand_dims(base, -1)
      ind_class = np.argmax(scores)
      y_pred.append(color_options[ind_class])
      y_true.append(row['color'])

  return y_true, y_pred

## Scoring model with Stroop probing technique

In [ ]:
y_true1, y_pred1 = score_model(feature_extractor, colors_orig_prompts)
accuracy_score(y_true1, y_pred1)

52it [08:15,  9.53s/it]


0.12884615384615383

In [ ]:
y_true1, y_pred1 = score_model(feature_extractor_large, colors_orig_prompts)
accuracy_score(y_true1, y_pred1)

52it [1:09:48, 80.55s/it]


0.057692307692307696

## Using the autoregressive property for predictions

In [ ]:
def retrieve_results(gen, llm_prompt, orig_prompts, max_new=1):
  y_true = []
  y_pred = []

  for index, row in df.iterrows():
      w = row['object']

      for p in orig_prompts:
        p = p.replace("WORD", w)
        input_prompt = p + llm_prompt
        out_p = gen(input_prompt, max_new_tokens=max_new, num_return_sequences=1, pad_token_id=50256)[0]['generated_text']
        out_p = out_p.replace(input_prompt, '').replace(' ', '')
        y_pred.append(out_p.lower())
        y_true.append(row['color'])

  return y_true, y_pred


## 1. instead of MASK, should be

In [ ]:
gpt_extra_prompt1 = '. instead of MASK, should be'

y_true1, y_pred1 = retrieve_results(generator, gpt_extra_prompt1, colors_orig_prompts)
accuracy_score(y_true1, y_pred1)

0.0057692307692307696

In [ ]:
y_true1[:5], y_pred1[:5]

(['grey', 'grey', 'grey', 'grey', 'grey'],
 ['just', 'taken', 'the', 'replaced', 'set'])

In [ ]:
gpt_extra_prompt1 = '. instead of MASK, should be'

y_true1, y_pred1 = retrieve_results(generator_large, gpt_extra_prompt1, colors_orig_prompts)
accuracy_score(y_true1, y_pred1)

0.01730769230769231

## 2. Empty. Use only two promts:

**"the basic shape of a WORD is",
"what is the shape of a WORD?"**

In [ ]:
last_two_shape_prompts = ["the basic color of a WORD is", "what is the color of a WORD?"]
gpt_extra_prompt2 = ''

retrieve_results(generator, gpt_extra_prompt2, last_two_shape_prompts)

GT: yellow : the basic color of a banana is 
result:   a 

GT: yellow : what is the color of a banana? 
result:  
 



In [ ]:
# With GPT2 large
last_two_shape_prompts = ["the basic color of a WORD is", "what is the color of a WORD?"]
gpt_extra_prompt2 = ''

retrieve_results(generator_large, gpt_extra_prompt2, last_two_shape_prompts)

GT: yellow : the basic color of a banana is 
result:   yellow 

GT: yellow : what is the color of a banana? 
result:   What 



In [ ]:
# With GPT2 large
last_two_shape_prompts = ["the basic color of a WORD is", "what is the shape of a WORD?"]
gpt_extra_prompt2 = ''

retrieve_results(generator_large, gpt_extra_prompt2, last_two_shape_prompts, max_new=2)

GT: yellow : the basic color of a banana is 
result:   the yellow 

GT: yellow : what is the shape of a banana? 
result:   This is 



## 3.  Fill in the MASK 

In [ ]:
gpt_extra_prompt3 = '.  Fill in the MASK'
y_true3, y_pred3 = retrieve_results(generator, gpt_extra_prompt3, colors_orig_prompts)
accuracy_score(y_true3, y_pred3)

0.0

In [ ]:
gpt_extra_prompt3 = '.  Fill in the MASK'
y_true3, y_pred3 = retrieve_results(generator_large, gpt_extra_prompt3, colors_orig_prompts)
accuracy_score(y_true3, y_pred3)

0.0019230769230769232

In [ ]:
y_true3[:3],  y_pred3[:3]

(['yellow', 'yellow', 'yellow'], [' with', ' as', ' details'])

In [ ]:
gpt_extra_prompt3 = '. MASK can be replaced with'
y_true3, y_pred3 = retrieve_results(generator, gpt_extra_prompt3, colors_orig_prompts)
accuracy_score(y_true3, y_pred3)

0.0

In [ ]:
gpt_extra_prompt3 = '. MASK can be replaced with'
y_true3, y_pred3 = retrieve_results(generator_large, gpt_extra_prompt3, colors_orig_prompts)
accuracy_score(y_true3, y_pred3)

0.013461538461538462

## 4. choose one option from the list: <> to replace MASK

In [ ]:
def retrieve_results_options(gen, orig_prompts):
  y_true = []
  y_pred = []

  for index, row in tqdm(df.iterrows()):
      w = row['object']

      for p in orig_prompts:
        p = p.replace("WORD", w)
        llm_prompt = '. Choose one word from: {} to replace MASK'.format(color_options)
        input_prompt = p + llm_prompt
        out_p = gen(input_prompt, max_new_tokens=1, num_return_sequences=1, pad_token_id=50256)[0]['generated_text']
        out_p = out_p.replace(input_prompt, '').replace(' ', '')
        y_pred.append(out_p.lower())
        y_true.append(row['color'])

  return y_true, y_pred


In [ ]:
y_true4, y_pred4 = retrieve_results_options(generator, colors_orig_prompts)
accuracy_score(y_true4, y_pred4)

52it [03:01,  3.49s/it]


0.0057692307692307696

In [ ]:
y_true4, y_pred4 = retrieve_results_options(generator_large, colors_orig_prompts)
accuracy_score(y_true4, y_pred4)

52it [18:29, 21.34s/it]


0.0

In [ ]:
5

5